In [1]:
import numpy as np
import pandas as pd
import os
import sys 
#import glob
import re
import nltk
from collections import Counter
nltk.download()  # Download text data sets, including stop words
from nltk.corpus import stopwords # Import the stop word list
from nltk.stem.porter import *
from bs4 import BeautifulSoup 


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [2]:
from gensim.models.keyedvectors import KeyedVectors
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [3]:
traindf=pd.read_csv('/home/harshit/Documents/PUCHO/labeledTrainData.tsv',sep='\t')
#unlabeledtraindf=pd.read_csv('/home/harshit/Documents/PUCHO/unlabeledTrainData.tsv', header=0, 
 #delimiter="\t", quoting=3 )
testdf=pd.read_csv('/home/harshit/Documents/PUCHO/testData.tsv',sep='\t')
#unlabeledtraindf.head()
#traindf.head(20)

In [202]:
#traindf.head(2)

In [4]:
def review_cleaner( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    return( " ".join( meaningful_words ))
    #return(sequence.pad_sequences(sentences, maxlen=1500))

In [5]:
# Get the number of reviews based on the dataframe column size
num_reviews = traindf["review"].size

num_testreviews=testdf["review"].size
# Initialize an empty list to hold the clean reviews
clean_train_reviews = []
clean_test_reviews = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in xrange( 0, num_reviews ):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    clean_train_reviews.append( review_cleaner( traindf["review"][i] ) )
for j in xrange( 0, num_testreviews ):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    clean_test_reviews.append( review_cleaner( testdf["review"][j] ) )

/usr/lib/python2.7/dist-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [195]:
#(clean_train_reviews[3])
#testdf["review"][4]
#clean_test_reviews[4]

In [6]:
def review_to_wordlist_for_word2vec( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    #count=0
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split() 
#     count+=len(words)
#     print(count)
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    #
    #stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    #m_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join(words ))  
    #return words

In [7]:
# Load the punkt tokenizer
# tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
# def review_to_sentences( review, tokenizer):#, remove_stopwords=False ):
#     # Function to split a review into parsed sentences. Returns a 
#     # list of sentences, where each sentence is a list of words
#     #
#     # 1. Use the NLTK tokenizer to split the paragraph into sentences
#     raw_sentences = tokenizer.tokenize(review.strip().decode("utf8"))
#     #
#     # 2. Loop over each sentence
#     bsentences = []
#     for raw_sentence in raw_sentences:
#         # If a sentence is empty, skip it
#         if len(raw_sentence) > 0:
#             bsentences.append( review_to_wordlist_for_word2vec( raw_sentence))
#     return bsentences
                             
    
#     # Return the list of sentences (each sentence is a list of words,
#     # so this returns a list of lists
                             

In [196]:
# sentences = []  # Initialize an empty list of sentences
# count=0
# print "Parsing sentences from training set"
# for review in traindf["review"]:
#     count=count+1
#     sentences += review_to_sentences(review, tokenizer)
# print(count)


In [26]:
#sentences[25002]

u'but with mark evading the law they realize that he needs to keep running'

In [198]:
# print "Parsing sentences from unlabeled set"
# count=0
# for review in unlabeledtraindf["review"]:
#     count=count+1
#     sentences += review_to_sentences(review, tokenizer)
# print(count)

In [76]:
# test_sentences=[]
# count=0
# print "Parsing sentences from test set"
# for review in testdf["review"]:
#     count=count+1
#     test_sentences += review_to_sentences(review, tokenizer)

Parsing sentences from test set


In [199]:
#(test_sentences[1:3])
#print count

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(nb_words=15000)
tokenizer.fit_on_texts(clean_train_reviews)
sequences = tokenizer.texts_to_sequences(clean_train_reviews)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

/usr/local/lib/python2.7/dist-packages/keras/preprocessing/text.py:157: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 74065 unique tokens.


In [71]:
len(sequences)

25000

In [99]:
#tokenizer.fit_on_texts(clean_test_reviews)


In [9]:
test_sequences = tokenizer.texts_to_sequences(clean_test_reviews)


In [10]:
#(sequences[6])
data = pad_sequences(sequences, maxlen=1500)

In [11]:
test_data=pad_sequences(test_sequences, maxlen=1500)

In [138]:
#len(data[27227])
#word_index['fawn']
#(word_index) 
#(sentences[1]) 
#len(sequences)
#sequences[1:3]
#test_data[3]
#sequences[3]


In [108]:
# # Import the built-in logging module and configure it so that Word2Vec 
# # creates nice output messages
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
#     level=logging.INFO)

# # Set values for various parameters
# num_features = 300    # Word vector dimensionality                      
# min_word_count = 30   # Minimum word count                        
# num_workers = 4       # Number of threads to run in parallel
# context = 10          # Context window size                                                                                    
# #negative=10
# downsampling = 1e-3   # Downsample setting for frequent words

# # Initialize and train the model (this will take some time)
# from gensim.models import word2vec
# print "Training model..."
# model = word2vec.Word2Vec(sentences, workers=num_workers, \
#             size=num_features, min_count = min_word_count, \
#             window = context, sample = downsampling)

# # If you don't plan to train the model any further, calling 
# # init_sims will make the model much more memory-efficient.
# model.init_sims(replace=True)

# # It can be helpful to create a meaningful model name and 
# # save the model for later use. You can load it later using Word2Vec.load()
# model_name = "300features_40minwords_10context"
# model.wv.save(model_name)

In [49]:
# Load the model that we created in Part 2
# from gensim.models import Word2Vec
# model=KeyedVectors.load(model_name)
#model = Word2Vec.load("300features_40minwords_10context")
# print(type(model.wv.syn0))
# (model.wv.syn0)#.shape)
#(model.wv.index2word)
# model.wv.syn0[1]
#model.wv.syn0['this']
#model['this']


2018-02-03 11:41:53,076 : INFO : loading EuclideanKeyedVectors object from 300features_40minwords_10context
2018-02-03 11:41:53,654 : INFO : setting ignored attribute syn0norm to None
2018-02-03 11:41:53,658 : INFO : loaded 300features_40minwords_10context


In [40]:
# index_dict={}
# for index,word in enumerate(model.wv.index2word):
#     index_dict[word] = index+1


#     #index_dict={word:index+1}
# #index_dict['this']
# #sorted(index_dict)
# #index_dict.items()
# len(index_dict)
# #model['this']

In [41]:
# word_vectors={}
# for word,index in index_dict.items():
#     word_vectors[word]=model[word]
# #len(word_vectors)


In [21]:
#model["movie"]
# Summarize review length of each sentence using X[i]
# print("Review length of traindata: ")
# result = [len(x) for x in traindf['review']]
# print("Mean %.2f words (%f)" % (np.mean(result), np.std(result)))
# print("Review length of unlabeledtraindf: ")
# result2 = [len(x) for x in unlabeledtraindf['review']]
# print("Mean %.2f words (%f)" % (np.mean(result2), np.std(result2)))
# print("Review length of testdata: ")
# result3 = [len(x) for x in testdf['review']]
# print("Mean %.2f words (%f)" % (np.mean(result3), np.std(result3)))
#std to check for distribution
#mean to calculate the average length of a review


# Review length of traindata: 
# Mean 1327.95 words (1005.530261)
# Review length of unlabeledtraindf: 
# Mean 1332.82 words (1007.186870)
# Review length of testdata: 
# Mean 1296.68 words (978.384307)

In [33]:
# model['this']

In [107]:
# def makeFeatureVec(words, model, num_features):
#     # Function to average all of the word vectors in a given
#     # paragraph
#     #
#     # Pre-initialize an empty numpy array (for speed)
#     featureVec = np.zeros((num_features,),dtype="float32")
#     #
#     nwords = 0
#     # 
#     # Index2word is a list that contains the names of the words in 
#     # the model's vocabulary. Convert it to a set, for speed 
#     index2word_set = set(model.wv.index2word)
#     #
#     # Loop over each word in the review and, if it is in the model's
#     # vocaublary, add its feature vector to the total
#     for word in words:
#         if word in index2word_set: 
#             nwords = nwords + 1.
#             featureVec = np.add(featureVec,model[word])
#     # 
#     # Divide the result by the number of words to get the average
#     featureVec = np.floor_divide(featureVec,nwords)
    
#     return featureVec


# def getAvgFeatureVecs(reviews, model, num_features):
#     # Given a set of reviews (each one a list of words), calculate 
#     # the average feature vector for each one and return a 2D numpy array 
#     # 
#     # Initialize a counter
#     counter = 0
#     # 
#     # Preallocate a 2D numpy array, for speed
#     reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
#     # 
#     # Loop through the reviews
#     for review in reviews:
#        #
#        # Print a status message every 1000th review
#         if counter%1000 == 0:
#             print "Review %d of %d" % (counter, len(reviews))
#        # 
#        # Call the function (defined above) that makes average feature vectors
#         reviewFeatureVecs[counter] = makeFeatureVec(review, model,num_features)
#        #
#        # Increment the counter
#         counter = (counter) + 1
#     return reviewFeatureVecs

In [106]:
# trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )
# testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

In [12]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import LSTM

In [73]:
len(word_index)

74065

In [200]:
# # assemble the embedding_weights in one numpy array
# vocab_dim = 7 # dimensionality of your word vectors
# n_symbols = len(word_index) + 1 # adding 1 to account for 0th index (for masking)
# embedding_weights = np.zeros((n_symbols, vocab_dim))
# for word,index in word_index.items():
#     embedding_weights[index, :] = data[index]

# # embedding_layer = Embedding(len(word_index) + 1,
# #                             EMBEDDING_DIM,
# #                             weights=[embedding_matrix],
# #                             input_length=MAX_SEQUENCE_LENGTH,
# #                             trainable=False)
# # define inputs here
# embedding_layer = Embedding(output_dim=vocab_dim, input_dim=n_symbols, trainable=True)
# embedding_layer.build((None,)) # if you don't do this, the next step won't work
# embedding_layer.set_weights([embedding_weights])

# #embedded = embedding_layer(input_layer)
# # ... continue model definition here

In [14]:
embedding_layer=Embedding(input_dim=len(word_index)+1, output_dim=64,input_length=1500)
modelk = Sequential()
modelk.add(embedding_layer)
modelk.add(LSTM(128))
modelk.add(Dropout(0.5))
modelk.add(Dense(1, activation='sigmoid'))

modelk.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# modelk.fit(x_train, y_train, batch_size=16, epochs=10)
# score = modelk.evaluate(x_test, y_test, batch_size=16)

In [75]:
# modelk = Sequential() # a layer model
# modelk.add(embedding_layer)
# #model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
# modelk.add(Flatten())
# modelk.add(Dense(250, activation='relu'))
# modelk.add(Dense(1, activation='sigmoid'))
# modelk.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(modelk.summary())
#len(data)

25000

In [15]:
X_train = data[:]
X_test=test_data[:]

In [144]:
test_data[3]

array([   0,    0,    0, ...,   23, 1090,   33], dtype=int32)

In [145]:
# Fit the model
modelk.fit(X_train, traindf["sentiment"], epochs=8,  verbose=2)
#modelk.fit(X,y, epochs=2,  verbose=2)
# Final evaluation of the model
#result = modelk.predict(testDataVecs, verbose=0)

Epoch 1/8
 - 1853s - loss: 0.3902 - acc: 0.8334
Epoch 2/8
 - 1839s - loss: 0.2387 - acc: 0.9097
Epoch 3/8
 - 1876s - loss: 0.2031 - acc: 0.9262
Epoch 4/8
 - 1871s - loss: 0.1782 - acc: 0.9356
Epoch 5/8
 - 1858s - loss: 0.1608 - acc: 0.9425
Epoch 6/8
 - 1851s - loss: 0.1444 - acc: 0.9495
Epoch 7/8
 - 1859s - loss: 0.1255 - acc: 0.9572
Epoch 8/8
 - 1878s - loss: 0.1069 - acc: 0.9644


In [27]:
#X_train[4]

In [16]:
# Test & extract results 
result = modelk.predict(X_test )


In [17]:
result

array([[0.4990441 ],
       [0.49804902],
       [0.50155693],
       ...,
       [0.5004143 ],
       [0.5019524 ],
       [0.5013185 ]], dtype=float32)

In [158]:
# result.shape
# testdf['id'].shape
# testdf['id'].type
# result.type
fresult=result.tolist()
ids=testdf['id'].tolist()
#len(fresult)

In [176]:
#(fresult.type())
#", ".join(fresult)
#fresult[0].type
flattened_list = [y for x in fresult for y in x]

In [186]:
#len(flattened_list)
binary_result = [round(x,0) for x in flattened_list]

In [190]:
# for i,value in enumerate(fresult):
#     if value>0.486:
#         fresult[i]=1
#     else:
#         fresult[i]=0
# (binary_result)
solution=[int(i) for i in binary_result]

In [192]:
#solution

In [193]:

d = {'id': ids, 'sentiment': solution}
output = pd.DataFrame(data=d)
# Write the test results 
#output = pd.DataFrame( data={"id":testdf["id"], "sentiment":result} )
output.to_csv( "Word2Vec_Keras_AverageVectors.csv", index=False, quoting=3 )

In [203]:
independent_dataset=pd.read_csv('/home/harshit/Documents/PUCHO/indDataset.tsv',sep='\t')

In [204]:
num_ind_reviews=independent_dataset["review"].size

ind_reviews = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in xrange( 0, num_ind_reviews ):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    ind_reviews.append( review_cleaner( independent_dataset["review"][i] ) )


In [209]:
ind_sequences = tokenizer.texts_to_sequences(ind_reviews)

In [210]:
ind_data=pad_sequences(ind_sequences, maxlen=1500)

In [214]:
ind_result=modelk.predict(ind_data)

In [216]:
#ind_result
ind_result_list=ind_result.tolist()
ids=independent_dataset['id'].tolist()

In [226]:
flattened_ilist = [y for x in ind_result_list for y in x]
binary_iresult = [round(x) for x in flattened_ilist]
isolution=[int(i) for i in binary_iresult]

In [231]:
# y_true=independent_dataset['sentiment']
# modelk.binary_accuracy(y_true,isolution)
isolution

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]